In [119]:
import numpy as np
import matplotlib.pyplot as plt
import os.path
import os
from scipy import odr
import time
from scipy.optimize import curve_fit
import circle_fit as cf
from sklearn.datasets import make_circles

%matplotlib qt

# import raw data

In [129]:
folder = "C:\\Users\\Florian\\Box Sync\\Projects"
project = "\\Chromium_RUS\\Individual scans\\up"
#project = "\\Chromium\\Individual scans\\down"
project = '/UTe2/RUS/UTe2_2007C/Red Pitaya/200818/monitor_2/individual scans'
#project = '/UTe2/RUS/UTe2_2007C/Red Pitaya/200819/frequency scans'
project = '\\UTe2\\RUS\\UTe2_2007C\\Red Pitaya\\201211' #4.04620K_950_1550_2.00000V_0.00000T_Dec11-2020__14-15-22_001_121120.bin

data_files = os.listdir(folder+project)
data_files = [i for i in data_files if i[-4:]=='.bin']
data_files = data_files
filenames = [folder + project + '\\' + i for i in data_files]

T = np.array([float(i[0: i.find('K')]) for i in data_files])
len(data_files)

16

In [130]:
T[7]

4.0462

In [131]:
filenames = [filenames[7]]
filenames

['C:\\Users\\Florian\\Box Sync\\Projects\\UTe2\\RUS\\UTe2_2007C\\Red Pitaya\\201211\\4.04620K_950_1550_2.00000V_0.00000T_Dec11-2020__14-15-22_001_121120.bin']

In [132]:
f = []
x = []
y = []

for i in filenames:
    dat = open(i, 'r')
    data = np.fromfile( dat, dtype=np.dtype('>f8') ) # the raw data are binary files, so this imports them
    split_index = int((len(data) - 1)/3)
    f.append(data[1 : split_index+1])
    x.append(data[split_index+1 : 2*split_index+1])
    y.append(data[2*split_index+1 : 3*split_index+1])

N = 1
M = 137
K = len(f)-50
f = np.array(f)#[M:K:N]
x = np.array(x)#[M:K:N]
y = np.array(y)#[M:K:N]

# f = [np.array(f)[2]]
# x = [np.array(x)[2]]
# y = [np.array(y)[2]]

# just plot the raw data

In [133]:
len(f)

1

In [134]:
f = np.array([f[i] for i in np.arange(len(f))])
x = np.array([x[i] for i in np.arange(len(x))])
y = np.array([y[i] for i in np.arange(len(y))])
plt.figure(figsize=(12,8))

for i in np.arange(len(f)):
    plt.plot(f[i], x[i]+0.01*i)

In [135]:
fflat = f.flatten()
xflat = x.flatten()
yflat = y.flatten()

In [136]:
mask = (fflat>1530.65) & (fflat<1531.03)
# mask = (fflat>968.2) & (fflat<968.86)
fmask = fflat[mask]
xmask = xflat[mask]
ymask = yflat[mask]

fitmask = (fmask > 1530.95) | (fmask < 1530.72)
# fitmask = (fmask > 968.83) | (fmask < 968.23)
ffitmask = fmask[fitmask]
xfitmask = xmask[fitmask]
yfitmask = ymask[fitmask]

xfit = np.polyval(np.polyfit(ffitmask, xfitmask, 9), fmask)
yfit = np.polyval(np.polyfit(ffitmask, yfitmask, 3), fmask)


plt.figure()
plt.plot(fmask, xmask, 'o-', zorder=0)
plt.plot(fmask, xfit, zorder=2)
plt.scatter(ffitmask, xfitmask, c='red', zorder=1)

plt.figure()
plt.plot(fmask, ymask, 'o-', zorder=0)
plt.plot(fmask, yfit, zorder=2)
plt.scatter(ffitmask, yfitmask, c='red', zorder=1)

C:\Program Files\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:3417: RankWarning: Polyfit may be poorly conditioned
  exec(code_obj, self.user_global_ns, self.user_ns)


In [148]:
real = xmask-xfit
imaginary = ymask-yfit

amp = np.sqrt( real**2 + imaginary**2 )
phase = np.arctan(imaginary/real)
phase = phase / max(phase) * max(amp)/2 + max(amp)/2

phasemask = (fmask>1530.78) & (fmask<1530.898)
plt.plot( fmask, amp, 'o-')
plt.plot(fmask[phasemask], phase[phasemask], 'o-')

In [164]:
def lorentzian(x, a, x0, b):
    return b / ((x-x0)**2 + a**2) / np.pi


guess = [0.03, 1530.8, 1]
# guess = [0.03, 968.517, 1]

lorentzian_mask = (fmask>1530.8) & (fmask<1530.89)
# lorentzian_mask = (fmask>968.3) & (fmask<968.75)

popt, pcov = curve_fit (lorentzian, fmask[lorentzian_mask], amp[lorentzian_mask], p0=guess)
fit = lorentzian(fmask, popt[0], popt[1], popt[2])

plt.plot(fmask, amp*1e3, 'o-')
plt.plot(fmask[lorentzian_mask], fit[lorentzian_mask]*1e3)
plt.xlabel('f (kHz)', fontsize=16)
plt.ylabel('Amplitude (mV)', fontsize=16)

Text(0, 0.5, 'Amplitude (mV)')

In [141]:
plt.figure(figsize=(6,6))
plt.scatter(xmask, ymask)

In [155]:
coordinates = np.array(list(zip(xmask, ymask)))
xc, yc, r, s = cf.hyper_fit(coordinates)

cf.plot_data_circle(xmask, ymask, xc, yc, r)

In [ ]:
maskf = np.array([(i>1890)&(i<1896) for i in f])
fp = np.array([f[i][maskf[i]] for i in np.arange(len(f))])
xp = np.array([x[i][maskf[i]] for i in np.arange(len(x))])
yp = np.array([y[i][maskf[i]] for i in np.arange(len(y))])
plt.figure(figsize=(12,8))

for i in np.arange(len(f)):
    plt.plot(fp[i], xp[i]+0.0002*i)

## single out what I think are resonances and just plot them

In [ ]:
w = 0
res = [2067-w, 2075+w, 2187.5-w, 2202.7+w, 2238-w, 2246+w, 2362-w, 2372+w, 2447.5-w, 2458+w]
mask = [(i<res[0]) | ((i>res[1]) & (i<res[2])) | ((i>res[3]) & (i<res[4])) | ((i>res[5]) & (i<res[6])) | ((i>res[7]) & (i<res[8])) | (i>res[9]) for i in f]
maskinv = np.invert(mask)
#mask = [(i>2005) & (i<2495) for i in f]

In [ ]:
maskres = [0,0,0,0,0]
maskres[0] = [(i>2060) & (i<2080) for i in f]
maskres[1] = [(i>2181) & (i<2208) for i in f]
maskres[2] = [(i>2231) & (i<2252) for i in f]
maskres[3] = [(i>2356) & (i<2380) for i in f]
maskres[4] = [(i>2441) & (i<2465) for i in f]


f1 = plt.figure(figsize=(12,8))
for mask in maskres:
    for i in np.arange(len(f)):
        plt.plot(f[i][mask[i]], y[i][mask[i]])

f2 = plt.figure(figsize=(12,8))
for mask in maskres:
    for i in np.arange(len(f)):
        plt.plot(f[i][mask[i]], x[i][mask[i]])

# fit and subtract a background from the raw data close to a resonance
## I couldn't fit the same background for all resonances (and all data files) at the same time so each resonance has to be dealt with individually

In [ ]:
# ffit = np.array([f[i][maskres[1][i]] for i in np.arange(len(f))])
# xfit = np.array([x[i][maskres[1][i]] for i in np.arange(len(f))])
# yfit = np.array([y[i][maskres[1][i]] for i in np.arange(len(f))])


maskres = np.array([(i<1892.5)|(i>1893.5) for i in fp])
ffit = np.array([fp[i][maskres[i]] for i in np.arange(len(fp))])
xfit = np.array([xp[i][maskres[i]] for i in np.arange(len(fp))])
yfit = np.array([yp[i][maskres[i]] for i in np.arange(len(fp))])

maskfit0 = [(i<2063) | (i>2075) for i in ffit]
maskfit1 = [(i<2185) | (i>2204) for i in ffit]
maskfit2 = [(i<2233) | (i>2249) for i in ffit]
maskfit3 = [(i<2359) | (i>2377) for i in ffit]
maskfit4 = [(i<2445) | (i>2462) for i in ffit]

In [ ]:
def fct (p, f):
    a = 0
    for i in np.arange(len(p)):
        a = a + p[i] * np.ones(len(f)) * f**i
    return a



initial_guess = 8*[0]
model = odr.Model(fct)
# maskfit = maskfit1
R = []

start_time = time.time()
for i in np.arange(len(ffit)):
    # datax = odr.RealData(ffit[i][maskfit[i]], xfit[i][maskfit[i]])
    # datay = odr.RealData(ffit[i][maskfit[i]], yfit[i][maskfit[i]])

    datax = odr.RealData(ffit[i], xfit[i])
    datay = odr.RealData(ffit[i], yfit[i])

    fitx = odr.ODR(datax, model, beta0=initial_guess)
    outx = fitx.run()
    poptx = outx.beta
    fity = odr.ODR(datay, model, beta0=initial_guess)
    outy = fity.run()
    popty = outy.beta

    xnew = xfit[i] - fct(poptx, ffit[i])
    ynew = yfit[i] - fct(popty, ffit[i])

    xnew = xp[i] - fct(poptx, fp[i])
    ynew = yp[i] - fct(popty, fp[i])

    R.append(xnew**2 + ynew**2)

# for i in np.arange(len(fp)):
#     datax = odr.RealData(fp[i], xp[i])
#     datay = odr.RealData(fp[i], yp[i])

#     fitx = odr.ODR(datax, model, beta0=initial_guess)
#     outx = fitx.run()
#     poptx = outx.beta
#     fity = odr.ODR(datay, model, beta0=initial_guess)
#     outy = fity.run()
#     popty = outy.beta

#     xnew = xp[i] - fct(poptx, fp[i])
#     ynew = yp[i] - fct(popty, fp[i])

#     R.append(xnew**2 + ynew**2)







R = np.array(R)
Rn = np.array([i/max(i) for i in R])

duration = time.time() - start_time
print ('it takes ' + str(round(duration,1)) + ' s')

# plot the amplitude vs frequency at different temperatures

In [ ]:
# plt.figure(figsize=(12,8))

# palette = plt.get_cmap('brg')
# for i in np.arange(len(ffit)):
#     plt.plot(ffit[i], Rn[i] + 0.01*i, label=str(T[i]), c=palette(i/(len(ffit)-1)), zorder=1-i )

# plt.legend()

In [ ]:
from matplotlib.collections import LineCollection

def multiline(xs, ys, c, ax=None, **kwargs):
    """Plot lines with different colorings

    Parameters
    ----------
    xs : iterable container of x coordinates
    ys : iterable container of y coordinates
    c : iterable container of numbers mapped to colormap
    ax (optional): Axes to plot on.
    kwargs (optional): passed to LineCollection

    Notes:
        len(xs) == len(ys) == len(c) is the number of line segments
        len(xs[i]) == len(ys[i]) is the number of points for each line (indexed by i)

    Returns
    -------
    lc : LineCollection instance.
    """

    # find axes
    ax = plt.gca() if ax is None else ax

    # create LineCollection
    segments = [np.column_stack([x, y]) for x, y in zip(xs, ys)]
    lc = LineCollection(segments, **kwargs)

    # set coloring of line segments
    #    Note: I get an error if I pass c as a list here... not sure why.
    lc.set_array(np.asarray(c))

    # add lines to axes and rescale 
    #    Note: adding a collection doesn't autoscalee xlim/ylim
    ax.add_collection(lc)
    ax.autoscale()
    return lc

In [ ]:
plotmask = np.array([(i>1892.5)&(i<1893.6) for i in fp])
Rplot = np.array([ Rn[i][plotmask[i]]+0.01*i for i in np.arange(len(Rn)) ])[0]
fplot = np.array( [ fp[i][plotmask[i]] for i in np.arange(len(fp)) ])[0]


plt.figure(figsize=(11,7))
plt.plot(fplot, Rplot, lw=2)


plt.xlabel('Frequency (kHz)', fontsize=20)
plt.ylabel('Amplitude (mV)', fontsize=20)
plt.tick_params(axis="both",direction="in", labelsize=20, bottom='True', top='True', left='True', right='True', 
                length=4, width=1, which = 'major')
#plt.ylim(-0.001, 0.07)
#plt.xlim(500, 3550)

In [ ]:
n = 4
plotmask = np.array([(i>1892.5)&(i<1893.6) for i in fp])
Rplot = np.array([ Rn[i][plotmask[i]]+0.01*i for i in np.arange(len(Rn))[::n] ])
fplot = np.array( [ fp[i][plotmask[i]] for i in np.arange(len(fp))[::n] ])


fig, ax = plt.subplots(figsize=(11,7))
# lc = multiline(ffit[::n], Rplot, T[::n], cmap='brg', lw=2)
lc = multiline(fplot, Rplot, T[::n], cmap='viridis', lw=2)


axcb = fig.colorbar(lc)
axcb.set_label('T (K)', fontsize=15)

plt.xlabel('Frequency (kHz)', fontsize=20)
plt.ylabel('Resonance Amplitude (a.u.)', fontsize=20)
plt.tick_params(axis="x",direction="in", labelsize=20, bottom='True', top='True', left='True', right='True', 
                length=4, width=1, which = 'major')
plt.tick_params(axis="y",direction="in", labelsize=0, bottom='True', top='True', left='False', right='False', length=0, width=0, which = 'major')


# extract the resonance frequency (as the maximum) and plot vs temperature

In [ ]:
index = np.array([abs(np.array(i) - 1) < 0.0000001 for i in Rn])
maxf = np.array([ffit[i][index[i]] for i in np.arange(len(index))])

initial_guess = 3*[0]
model = odr.Model(fct)
resonances = np.zeros(len(maxf))

start_time = time.time()
for i in np.arange(len(maxf)):
    mask = (ffit[i]>maxf[i]-2) & (ffit[i]<maxf[i]+2)
    data = odr.RealData(ffit[i][mask], Rn[i][mask])
    
    fit = odr.ODR(data, model, beta0=initial_guess)
    out = fit.run()
    popt = out.beta

    resonances[i] = (-popt[1]/2/popt[2])

duration = time.time() - start_time
print ('it takes ' + str(round(duration,1)) + ' s')

In [ ]:
plt.figure(figsize=(11,7))
plt.plot(T, resonances)
plt.xlabel('Temperature (K)', fontsize=20)
plt.ylabel('Resonance Frequency (kHz)', fontsize=20)
plt.tick_params(axis="both",direction="in", labelsize=20, bottom='True', top='True', left='True', right='True', 
                length=4, width=1, which = 'major')